In [ ]:
import logging
logging.basicConfig(level=logging.INFO)

import pickle 
import numpy as np
import os
from tqdm import tqdm

from openfold.data import data_pipeline, feature_pipeline
from openfold.utils.tensor_utils import tensor_tree_map, dict_multimap
from openfold.config import model_config
from openfold.utils.loss import compute_contact_ca

In [ ]:
ss_file = '/Users/chenceshi/Downloads/Chrome Downloads/ss_annotation_31885.pkl'
ss_dict = {}
with open(ss_file, 'rb') as fin:
    second_structure_data = pickle.load(fin)
logging.warning(f"get {len(second_structure_data)} second structure data")

# for ss_ in second_structure_data:
#     tag_ = ss_['tag']
#     chain_ = ss_['chain']
#     assert len(set(chain_)) == 1
#     if chain_[0] != tag_[4]:
#         logging.warning(f"remove bad pdbs: {tag_}, found chain: {chain_}")
#         continue
#     tag2seq[ss_['tag']] = ss_['sequence']
#     tag2top[ss_['tag']] = ss_['topology']
        
for ss in second_structure_data:
    ss_dict[ss['tag']] = ss['ss3']
my_data_pipeline = data_pipeline.DataPipeline(ss_dict)
config = model_config(name='initial_training', train=True)
my_feature_pipeline = feature_pipeline.FeaturePipeline(config.data)

In [ ]:
# all bad pdbs
for x in second_structure_data:
    tag = x['tag']
    chain = x['chain']
    assert len(set(chain)) == 1
    if chain[0] != tag[4]:
        print(tag)

In [ ]:
fname = '1afb100'
path = '/Users/chenceshi/Downloads/1afb100.pdb'
data = my_data_pipeline.process_pdb(path, fname[4])
feats = my_feature_pipeline.process_features(data, 'train')

In [ ]:
feats['all_atom_positions'].shape

In [ ]:
cycle_no = 0
fetch_cur_batch = lambda t: t[..., cycle_no]
cur_feats = tensor_tree_map(fetch_cur_batch, feats)

contact = compute_contact_ca(
    cur_feats["all_atom_positions"],
    cur_feats["all_atom_mask"],
    cutoff=12
)
contact_ = contact[:63,:63].numpy()

In [ ]:
beautiful_out = []
for line in contact_:
    tmp = []
    for x_ in line:
        tmp.append(str(x_))
    beautiful_out.append(''.join(tmp))
print('\n'.join(beautiful_out))
